In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# import os

# os.chdir('drive/MyDrive/three-phase-fidelity-wip')
# print(os.getcwd())

# # # # !pip install -r requirements.txt
# !pip install ipython==7.31.1
# !pip install importlib-metadata==4.10.0
# !pip install acv-exp
# !pip install hyperopt==0.27
# # # ## !pip freeze > requirements.txt

# !pip install lime
# !pip install shap
# !pip install hyperopt

In [2]:
import pandas as pd
import numpy as np

from DatasetManager import DatasetManager

from sklearn.metrics import mean_absolute_percentage_error, accuracy_score, mean_squared_error, r2_score
import sklearn

import os
import joblib

import warnings
warnings.filterwarnings('ignore')

import scipy

import shap
import lime
import learning
import pyAgrum
#from acv_explainers import ACXplainer

#from anchor import anchor_tabular

import json

from tqdm import tqdm, tqdm_notebook

from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

from collections import Counter
import seaborn as sns

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# def get_shap_vals(explainer, instance, cls, classification, exp_iter, feat_list):
    
#     shap_exp = []
    
#     pred = cls.predict(instance.reshape(1, -1))
    
#     for i in range(exp_iter):
#         if type(explainer) == shap.explainers._tree.Tree:
#             exp = explainer(instance, check_additivity = False).values
#         else:
#             exp = explainer(instance.reshape(1, -1)).values
                
#         if exp.shape == (1, len(feat_list), 2):
#             exp = exp[0]
            
#         #print(exp.shape)
        
#         if exp.shape == (len(feat_list), 2):
#             exp = np.array([feat[pred] for feat in exp]).reshape(len(feat_list))
#         elif exp.shape == (1, len(feat_list)) or exp.shape == (len(feat_list), 1):
#             exp = exp.reshape(len(feat_list))
            
#         shap_exp.append(exp)
        
        
#     if np.array(shap_exp).shape != (exp_iter, len(feat_list)):
#         raise Exception("Explanation shape is not correct. It is", np.array(shap_exp).shape, "instead of the expected", (exp_iter, len(feat_list)))
            
#     avg_val = np.average(shap_exp, axis = 0)
#     abs_val = [abs(val) for val in avg_val]
    
#     return avg_val, abs_val

In [4]:
# def get_lime_features(explainer, instance, cls, classification, exp_iter, feat_list):
#     lime_exp = []
    
#     for i in range(exp_iter):
#         if classification==True:
#             lime_exp.extend(explainer.explain_instance(instance, cls.predict_proba, 
#                                                 num_features=len(feat_list), labels=[0,1]).as_list())
#         else:
#             lime_exp.extend(explainer.explain_instance(instance, cls.predict, 
#                                                 num_features=len(feat_list), labels=[0,1]).as_list())
            
#     weights = [[] for each in feat_list]
#     for exp in lime_exp:
#         feat = exp[0]
#         if '<' in feat:
#             feat = exp[0].replace("= ",'')
#             parts = feat.split('<')
#         elif '>' in feat:
#             feat = exp[0].replace("= ",'')
#             parts = feat.split('>')
#         else:
#             parts = feat.split("=")
        
#         for part in parts:
#             if part.replace('.','').replace(' ','').isdigit()==False:
#                 feat_name = part.replace(' ','')
#         n = feat_list.index(feat_name)
#         weights[n].append(exp[1])
    
#     weights = np.transpose(weights)
#     avg_weight = np.average(np.array(weights), axis = 0)
#     abs_weight = [abs(weight) for weight in avg_weight]
    
#     bins = pd.cut(abs_weight, 4, retbins = True, duplicates = "drop")
#     q1_min = bins[1][-2]
    
#     sorted_weight = np.copy(abs_weight)
#     sorted_weight.sort()
    
#     lime_features = [i for i in range(len(feat_list)) if abs_weight[i] >= q1_min]
    
#     return avg_weight, abs_weight

In [5]:
# def get_linda_features(instance, cls, scaler, dataset, exp_iter, feat_list):
#     label_lst = ["Negative", "Positive"]
    
#     feat_pos = []
#     lkhoods = []
    
#     for i in range(exp_iter):
#         [bn, inference, infoBN] = learning.generate_BN_explanations(instance, label_lst, feat_list, "Result", 
#                                                                        None, scaler, cls, save_to+"/"+cls_method+"/", dataset, show_in_notebook = False)
        
#         ie = pyAgrum.LazyPropagation(bn)
#         result_posterior = ie.posterior(bn.idFromName("Result")).topandas()
#         result_proba = result_posterior.loc["Result", label_lst[instance['predictions']]]
#         row = instance['original_vector']
#         #print(row)

#         likelihood = [0]*len(feat_list)

#         for j in range(len(feat_list)):
#             var_labels = bn.variable(feat_list[j]).labels()
#             str_bins = list(var_labels)
#             bins = []

#             for disc_bin in str_bins:
#                 disc_bin = disc_bin.strip('"(]')
#                 cat = [float(val) for val in disc_bin.split(',')]
#                 bins.append(cat)

#             for k in range(len(bins)):
#                 if k == 0 and row[j] <= bins[k][0]:
#                     feat_bin = str_bins[k]
#                 elif k == len(bins)-1 and row[j] >= bins[k][1]:
#                     feat_bin = str_bins[k]
#                 elif row[j] > bins[k][0] and row[j] <= bins[k][1]:
#                     feat_bin = str_bins[k]

#             ie = pyAgrum.LazyPropagation(bn)
#             ie.setEvidence({feat_list[j]: feat_bin})
#             ie.makeInference()
            
#             result_posterior = ie.posterior(bn.idFromName("Result")).topandas()
#             new_proba = result_posterior.loc["Result", label_lst[instance['predictions']]]
#             #print(result_proba, new_proba)
#             proba_change = result_proba-new_proba
#             likelihood[j] = abs(proba_change)

#         lkhoods.append(likelihood)
        
#     bins = pd.cut(np.mean(lkhoods, axis=0), 4, retbins = True, duplicates = "drop")
#     q1_min = bins[1][-2]

#     #If fixing all features produces the same result for the class,
#     #return all features
#     if len(set(np.mean(lkhoods, axis=0)))==1:
#         feat_pos.extend(range(len(feat_list)))
#         #print(lkhoods)
#     else:
#         feat_pos.extend(list(np.where(np.mean(lkhoods, axis=0) >= q1_min)[0]))

#     feat_pos = set(feat_pos)
#     #print(feat_pos)
    
#     return np.mean(lkhoods, axis=0), np.mean(lkhoods, axis=0)

In [6]:
# def get_acv_features(explainer, instance, cls, X_train, y_train, exp_iter, feat_list):
#     instance = instance.reshape(1, -1)
#     y = cls.predict(instance)

#     feat_pos = []
#     feat_imp = []

#     for i in range(exp_iter):
#         sdp_importance, sdp_index, size, sdp = explainer.importance_sdp_rf(instance, y, X_train, y_train)
#         feat_pos.extend(sdp_index[0, :size[0]])
        
#         sufficient_expl, sdp_expl, sdp_global = explainer.sufficient_expl_rf(instance, y, X_train, y_train)
#         #print(np.array(sufficient_expl).shape)
#         lximp = explainer.compute_local_sdp(len(feat_list), sufficient_expl[0])
#         feat_imp.append(lximp)


#     feats = Counter(feat_pos)
#     imp = feats.items()

#     occ = np.zeros(len(feat_list))

#     for each in imp:
#         occ[each[0]] = each[1]
        
#     avg_imp = np.mean(feat_imp, axis=0)
#     abs_imp = [abs(imp) for imp in avg_imp]

#     print("Frequency of occurrence:", occ)
#     #print("Feature importance score:", avg_imp)
#     return occ, occ
#     #return avg_imp, abs_imp

In [7]:
# def get_anchor_features(explainer, instance, cls, train_data, classification, exp_iter):
#     anchor_exp = []
#     for i in range(exp_iter):
#         if classification == True:
#             anchor_exp.extend(explainer.explain_instance(instance, cls.predict).names())
#     print(anchor_exp)

#     locs = []
#     for exp in anchor_exp:
#         feat = exp.replace("= ",'')
#         #print(feat)
#         if '<' in feat:
#             parts = feat.split('<')
#             #print("less than", parts)
#         elif '>' in feat:
#             parts = feat.split('>')
#             #print("more than", parts)

#         for part in parts:
#             if part.replace('.','').replace(' ','').isdigit()==False:
#                 feat_name = part.replace(' ','')
#         locs.append(feat_list.index(feat_name))
    
#     feats = Counter(locs)
#     imp = feats.items()
    
#     #print(imp)
    
#     occ = np.zeros(len(feat_list))
    
#     for each in imp:
#         occ[each[0]] = each[1]
        
#     print("Frequency of occurrence:", occ)
    
#     return occ, occ

In [8]:
# def get_explanation_features(explainer, instance, cls, scaler, dataset, classification, exp_iter, xai_method, feat_list, X_train, y_train):
#     if xai_method == "SHAP":
#         feat_pos = get_shap_vals(explainer, instance, cls, classification, exp_iter, feat_list)
        
#     elif xai_method == "LIME":
#         feat_pos = get_lime_features(explainer, instance, cls, classification, exp_iter, feat_list)
        
#     elif xai_method == "LINDA":
#         feat_pos = get_linda_features(instance, cls, scaler, dataset, exp_iter, feat_list)

#     elif xai_method == "ACV":
#         feat_pos = get_acv_features(explainer, instance, cls, X_train, y_train, exp_iter, feat_list)
        
#     elif xai_method == "Anchor":
#         feat_pos = get_anchor_features(explainer, instance, cls, X_train, classification, exp_iter)
    
#     return feat_pos

In [9]:
def get_tree_features(cls, instance):
    tree = cls.tree_
    lvl = 0
    left_child = tree.children_left[lvl]
    right_child = tree.children_right[lvl]

    feats = []
    
    while left_child != sklearn.tree._tree.TREE_LEAF and right_child != sklearn.tree._tree.TREE_LEAF:
        feature = tree.feature[lvl]
        feats.append(feature)
        
        if instance[feature] < tree.threshold[lvl]:
            lvl = left_child
        else:
            lvl = right_child
            
        left_child = tree.children_left[lvl]
        right_child = tree.children_right[lvl]
            
            
    feat_pos = np.zeros(len(instance))
    n = len(feats)
    for i in feats:
        feat_pos[i]+=n
#        feat_pos[i]+=1
        n=n-1
    #feat_pos = set(feats)
    
    return feat_pos

In [10]:
def get_reg_features(cls):

    og_coef = cls.coef_
    if len(og_coef.shape) > 1:
        og_coef = og_coef[0]
    
    coef = [abs(val) for val in og_coef]
    
#     bins = pd.cut(coef, 4, retbins = True, duplicates = "drop")
#     q1_min = bins[1][-2]
    
#     feat_pos = [i for i in range(len(coef)) if coef[i] > q1_min]
    
    return coef

In [11]:
def get_nb_features(cls, instance):
    pred = cls.predict(instance.reshape(1, -1))
    means = cls.theta_[pred][0]
    std = np.sqrt(cls.var_[pred])[0]
    
    alt = 1-pred
    alt_means = cls.theta_[alt][0]
    alt_std = np.sqrt(cls.var_[alt])[0]

    likelihoods = []
    
    for i in range(len(means)):
        lk = scipy.stats.norm(means[i], std[i]).logpdf(instance[i])
        alt_lk = scipy.stats.norm(alt_means[i], alt_std[i]).logpdf(instance[i])
        lkhood = lk-alt_lk
#        lkhood = scipy.stats.norm(means[i], std[i]).logpdf(instance[i])
        likelihoods.append(lkhood)
    
#     bins = pd.cut(likelihoods, 4, retbins = True, duplicates = "drop")[1]
#     lim = bins[-2]
    
# #     bins = pd.cut(likelihoods, 10, retbins = True, duplicates = "drop")[1]
# #     lim_1 = bins[-2]
# #     lim_2 = bins[1]
    
# #     sortedls = sorted(likelihoods, reverse=True)
# #     pos = math.ceil(len(likelihoods)/4)
# #     lim = likelihoods[pos]
    
#     feat_pos = [i for i in range(len(likelihoods)) if likelihoods[i] >= lim]# or likelihoods[i] <= lim_2]
    
    return np.abs(likelihoods)

In [12]:
def get_true_rankings(cls, instance, cls_method, X_train, feat_list):
    if cls_method == "decision_tree":
        feat_pos = get_tree_features(cls, instance)
        
    elif cls_method == "logit" or cls_method == "lin_reg":
        feat_pos = get_reg_features(cls)
        
    elif cls_method == "nb":
        feat_pos = get_nb_features(cls, instance)
        
    return feat_pos

In [13]:
def permute_instance(instance, i, perm_iter = 100, min_i = 0, max_i=1, mean_i=0, mode="permutation"):
    if mode=="baseline_max":
        n_val = [max_i]*perm_iter
    elif mode=="baseline" or mode=="baseline_mean":
        n_val = [mean_i]*perm_iter
    elif mode=="baseline_min":
        n_val = [min_i]*perm_iter
    elif mode=="baseline_0":
        n_val = [0]*perm_iter
    else:
        n_val = np.random.uniform(min_i, max_i, perm_iter)
    
    permutations = np.array([instance]*perm_iter).transpose()
    permutations[i] = n_val
    permutations = permutations.transpose()

    return permutations

In [14]:
def cycle_values(instance, i, perm_iter = 100, min_i = 0, max_i=1, mean_i=0, unique_values=[0,1], mode="permutation"):
    if mode=="baseline_max":
        n_val = [max_i]*perm_iter
    elif mode=="baseline" or mode=="baseline_mean":
        n_val = [mean_i]*perm_iter
    elif mode=="baseline_min":
        n_val = [min_i]*perm_iter
    elif mode=="baseline_0":
        n_val = [0]*perm_iter
    else:
        n_val = np.random.choice(unique_values, perm_iter)
    
    permutations = np.array([instance]*perm_iter).transpose()
    permutations[i] = n_val
    permutations = permutations.transpose()

    return permutations

In [15]:
# path to project folder
# please change to your own
PATH = os.getcwd()

dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "prefix"
cls_encoding = "index"
cls_method = "nb"

classification=True

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)
save_to = os.path.join(PATH, dataset_ref, cls_method, method_name)

modes = ["permutation", "baseline_min", "baseline_mean", "baseline_max", "baseline_0"]

sample_size = 2
exp_iter = 10
max_feat = 10
max_prefix = 20
random_state = 22
perm_iter = 1000

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"],
    #"insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1"],# "sepsis_cases_2", "sepsis_cases_4"]
    "production": ["production"] 
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [16]:
# # path to project folder
# # please change to your own
# PATH = os.getcwd()

# dataset = "nursery"
# cls_method = "nb"

# classification = True
# # xai_method = "SHAP"

# modes = ["permutation", "baseline_min", "baseline_mean", "baseline_max", "baseline_0"]

# random_state = 22
# exp_iter = 10
# perm_iter = 1000

# save_to = "%s/%s/" % (PATH, dataset)
# dataset_folder = "%s/datasets/" % (save_to)
# final_folder = "%s/%s/" % (save_to, cls_method)

# #Get datasets
# X_train = pd.read_csv(dataset_folder+dataset+"_Xtrain.csv", index_col=False, sep = ";")
# y_train = pd.read_csv(dataset_folder+dataset+"_Ytrain.csv", index_col=False, sep = ";")
# test_x = pd.read_csv(final_folder+"test_sample.csv", index_col=False, sep = ";").values
# results = pd.read_csv(os.path.join(final_folder,"results.csv"), index_col=False, sep = ";")
# actual = results["Actual"].values

# with open(dataset_folder+"col_dict.json", "r") as f:
#     col_dict = json.load(f)
# f.close()

# feat_list = [each.replace(' ','_') for each in X_train.columns]

# cls = joblib.load(save_to+cls_method+"/cls.joblib")
# scaler = joblib.load(save_to+"/scaler.joblib")

In [17]:
# if xai_method == "SHAP":
#   #  explainer = shap.explainers._permutation.Permutation(cls.predict_proba, X_train)
#     if cls_method == "xgboost" or cls_method == "decision_tree":
#         explainer = shap.Explainer(cls)
#     elif cls_method == "nb":
#         if classification:
#             masker = shap.maskers._tabular.Independent(X_train.values, len(X_train))
#             explainer = shap.Explainer(cls.predict_proba, masker)
#         else:
#             print("NB is classification only")
#     elif cls_method == "logit" or cls_method == "lin_reg":
#         masker = shap.maskers._tabular.Independent(X_train.values, len(X_train))
#         explainer = shap.Explainer(cls, masker)
#     else:
#         explainer = shap.Explainer(cls, X_train)
#     print(type(explainer))
    
# elif xai_method == "LIME":
#     if col_dict['discrete'] != None:
#         cat_cols = [each.replace(' ','_') for each in col_dict['discrete']]
#         col_inds = [feat_list.index(each) for each in cat_cols]
#     else:
#         col_inds = []
    
#     if classification==True:
#         class_names=['Negative','Positive']# negative is 0, positive is 1, 0 is left, 1 is right
#         explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names = feat_list, 
#                                                             class_names=class_names, categorical_features = col_inds,
#                                                             discretize_continuous=False)
#     else:
#         class_names = ['Final Value']
#         explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names = feat_list, 
#                                                            class_names=class_names, discretize_continuous=True, 
#                                                            categorical_features = col_inds, mode = "regression")
        
# elif xai_method == "LINDA":
#     test_dict = learning.generate_local_predictions( test_x, results["Actual"].values, cls, scaler, None )
# #    feat_list = feat_list+["Result"]

#     explainer = None

# elif xai_method == "ACV":
#     explainer = joblib.load(save_to+cls_method+"/acv_explainer.joblib")
    
# elif xai_method == "Anchor":
#     if classification:
#         class_names=['Negative','Positive']
#         explainer = anchor_tabular.AnchorTabularExplainer(class_names, feat_list, X_train.values)
#     else:
#         raise Exception("Anchor only works for classification")

In [18]:
# if cls_method == "nb":
    
#     probas = cls.predict_proba(test_x)[:, 1]
    
#     clf_isotonic = CalibratedClassifierCV(cls, cv="prefit", method="isotonic")
#     clf_isotonic.fit(X_train, y_train)
#     isotonic_probas = clf_isotonic.predict_proba(test_x)[:, 1]
    
#     clf_sigmoid = CalibratedClassifierCV(cls, cv="prefit", method="sigmoid")
#     clf_sigmoid.fit(X_train, y_train)
#     sigmoid_probas = clf_sigmoid.predict_proba(test_x)[:, 1]
    
#     cls_score = brier_score_loss(results["Actual"], probas)
#     iso_score = brier_score_loss(results["Actual"], isotonic_probas)
#     sig_score = brier_score_loss(results["Actual"], sigmoid_probas)
    
#     if iso_score < sig_score and iso_score < cls_score:
#         print("Winner is iso")
#         cls = clf_isotonic.calibrated_classifiers_[0].base_estimator
#     elif sig_score < iso_score and sig_score < cls_score:
#         print("Winner is sigmoid")
#         cls = clf_sigmoid.calibrated_classifiers_[0].base_estimator
#     else:
#         cls = cls

In [19]:
# min_X = np.min(X_train)
# max_X = np.max(X_train)
# mean_X = np.mean(X_train, axis=0)
# unique_values = pd.Series({col: X_train[col].unique() for col in X_train.columns})

In [20]:
# if X_train.columns[-3] in col_dict["continuous"]:
#     print(True)
# else:
#     print(False)

In [21]:
all_results = []

for dataset_name in datasets:
        
    for ii in range(n_iter):
        num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s/%s/pipelines'% 
                                                                    (dataset_ref, cls_method, method_name)))])
        dataset_manager = DatasetManager(dataset_name)
        

        for bucket in tqdm_notebook(range(num_buckets)):
            bucketID = bucket+1
            print ('Bucket', bucketID)

            #import everything needed to sort and predict
            pipeline_path = os.path.join(PATH, save_to, "pipelines/pipeline_bucket_%s.joblib" % 
                                         (bucketID))
            pipeline = joblib.load(pipeline_path)
            feature_combiner = pipeline['encoder']
            if 'scaler' in pipeline.named_steps:
                scaler = pipeline['scaler']
            else:
                scaler = None
            cls = pipeline['cls']

            #import training data for bucket
            trainingdata = pd.read_csv(os.path.join(PATH, "%s/%s/%s/train_data/train_data_bucket_%s.csv" % 
                                                          (dataset_ref, cls_method, method_name, bucketID)))
            targets = pd.read_csv(os.path.join(PATH, "%s/%s/%s/train_data/y_train_bucket_%s.csv" % 
                                                          (dataset_ref, cls_method, method_name, bucketID))).values
            
            #Identify feature names
            feat_list = [feat.replace(" ", "_") for feat in feature_combiner.get_feature_names()]
            cats = [feat for col in dataset_manager.dynamic_cat_cols+dataset_manager.static_cat_cols 
                for feat in range(len(feat_list)) if col in feat_list[feat]]
            cat_cols = trainingdata.columns[cats]
            
            #scale data if necessary
            if scaler != None:
                trainingdata = scaler.transform(trainingdata)
                trainingdata = pd.DataFrame(trainingdata, columns=feat_list)

            #find relevant samples for bucket
            sample_instances = pd.read_csv(os.path.join(PATH, "%s/%s/%s/samples/test_sample_bucket_%s.csv" % 
                                      (dataset_ref, cls_method, method_name, bucketID))).values
            results = pd.read_csv(os.path.join(PATH, "%s/%s/%s/samples/results_bucket_%s.csv" % 
                                      (dataset_ref, cls_method, method_name, bucketID)))

            #scale data if necessary
            if scaler != None:
                sample_instances = scaler.transform(sample_instances)
            

            #trainingdata = pd.DataFrame(trainingdata, columns=feat_list)
            min_X = np.min(trainingdata)
            max_X = np.max(trainingdata)
            mean_X = np.mean(trainingdata, axis=0)
            unique_values = pd.Series({col: trainingdata[col].unique() for col in trainingdata.columns})

            for mode in modes:
                ktb_list = []
                true_v_mape = []
                true_v_rmse = []
                true_v_r2 = []

                #for i in tqdm_notebook(range(len(sample_instances))):
                for i in range(len(sample_instances)):
                    instance = sample_instances[i]

                    tr = get_true_rankings(cls, instance, cls_method, trainingdata, feat_list)

                    if classification:
                        pred = cls.predict(instance.reshape(1, -1))
                        proba = cls.predict_proba(instance.reshape(1, -1)).reshape(2)[pred]
                #        p1_list = [pred]*perm_iter
                        p1_list = list(proba)*perm_iter
            #         else:
            #             pred = cls.predict(instance.reshape(1, -1)).reshape(1)
            #             p1_list = [pred]*perm_iter

                #     pred = cls.predict(instance.reshape(1, -1)).reshape(1)
                #     p1_list = [pred]*perm_iter

                    perm_mape = np.zeros(len(instance))
                    perm_rmse = np.zeros(len(instance))
                    perm_r2 = np.zeros(len(instance))

                    for j in range(len(instance)):
                       # print("Permuting", feat_list[j])

                        if cats != None:
                            if trainingdata.columns[j] in cat_cols:
                                permutations = cycle_values(instance, j, perm_iter, min_X[j], max_X[j], mean_X[j], unique_values[trainingdata.columns[j]], mode)
                            else:
                                permutations = permute_instance(instance, j, perm_iter, min_X[j], max_X[j], mean_X[j], mode)
                        else:
                            permutations = permute_instance(instance, j, perm_iter, min_X[j], max_X[j], mean_X[j], mode)

                        if classification:
                #             p2_list = cls.predict_proba(permutations)[:, pred].reshape(perm_iter)
                #             perm_mape[j] = mean_absolute_percentage_error(p1_list, p2_list)

                            p2_list = cls.predict_proba(permutations).transpose()[pred].reshape(perm_iter)
                       #     perm_acc[j] = 1-accuracy_score(p1_list, p2_list)
                            perm_mape[j] = mean_absolute_percentage_error(p1_list, p2_list)
                            perm_rmse[j] = mean_squared_error(p1_list, p2_list, squared=False)
                            perm_r2[j] = r2_score(p1_list, p2_list)

                #             perm_mape[j] = len([np for np in new_preds if np!=pred])
                #             print(pred)
                #             print(new_preds)
                #             print(perm_mape[j])
            #            else:
            #                 p2_list = cls.predict(permutations).reshape(perm_iter)
            #                 perm_mape[j] = mean_absolute_percentage_error(p1_list, p2_list)

                        #p2_list = cls.predict(permutations).reshape(perm_iter)
                        #perm_mape[j] = mean_absolute_percentage_error(p1_list, p2_list)

                    #print("Final MAPE for all features:", perm_mape)
                    mape_corr = scipy.stats.kendalltau(tr, perm_mape, variant="b")[0]
                    rmse_corr = scipy.stats.kendalltau(tr, perm_rmse, variant="b")[0]
                    r2_corr = scipy.stats.kendalltau(tr, perm_r2, variant="b")[0]

                    #if xai_method=="LINDA":
                    #    instance = test_dict[i]

                    #avg_explanation, abs_explanation = get_explanation_features(explainer, instance, cls, scaler, dataset, classification, exp_iter, xai_method, 
                    #                                                        feat_list, X_train.values, y_train)
                    #print("Average explanation:", abs_explanation)

                    #if np.all(abs_explanation==abs_explanation[0]):
                    #    ktb=0
                    #else:
                    #    ktb = scipy.stats.kendalltau(perm_mape, abs_explanation, variant="b")[0]

                  #  ktb_list.append(ktb)
                    true_v_mape.append(mape_corr)
                    true_v_rmse.append(rmse_corr)
                    true_v_r2.append(r2_corr)
                    
#                     print(tr)
#                     print(perm_r2)
#                     print(r2_corr)

                #results[xai_method+"_KT-B"] = ktb_list
                results["MAPE Correctness"] = true_v_mape
                results["RMSE Correctness"] = true_v_rmse
                results["R2 Correctness"] = true_v_r2
                results["Mode"] = [mode]*results.shape[0]
                print(mode, "Mean correlation:", np.mean(results["R2 Correctness"]))
                results.to_csv(os.path.join(save_to,"samples", mode+"_results_bucket_%s.csv" % (bucketID)), 
                               index = False, sep = ";")
                all_results.append(results)
                
pd.concat(all_results).to_csv(os.path.join(PATH,"%s/%s/%s/samples/all_perm_results.csv") % (dataset_ref, cls_method, method_name), 
                               sep=";", index=False)

  0%|          | 0/14 [00:00<?, ?it/s]

Bucket 1
permutation Mean correlation: -0.8138038460194186
baseline_min Mean correlation: -0.6924256511258209
baseline_mean Mean correlation: -0.7693594015749742
baseline_max Mean correlation: -0.8026927349083075
baseline_0 Mean correlation: -0.6924256511258209
Bucket 2
permutation Mean correlation: -0.6634578127217019
baseline_min Mean correlation: -0.5745391324717293
baseline_mean Mean correlation: -0.6731843575418994
baseline_max Mean correlation: -0.659112346863032
baseline_0 Mean correlation: -0.5745391324717293
Bucket 3
permutation Mean correlation: -0.6784444054636882
baseline_min Mean correlation: -0.4143929226952225
baseline_mean Mean correlation: -0.5299989644044821
baseline_max Mean correlation: -0.5594801025448266
baseline_0 Mean correlation: -0.4143929226952225
Bucket 4
permutation Mean correlation: -0.540529203507564
baseline_min Mean correlation: 0.1414291785013539
baseline_mean Mean correlation: -0.3856603310893573
baseline_max Mean correlation: -0.4492703997775157
base

In [22]:
trainingdata.columns[j]

'index__Resource_13_other'

In [23]:
trainingdata.columns[cats]

Index(['index__Activity_0_A_SUBMITTED-COMPLETE',
       'index__Activity_1_A_PARTLYSUBMITTED-COMPLETE',
       'index__Activity_2_A_PREACCEPTED-COMPLETE',
       'index__Activity_2_W_Afhandelen_leads-SCHEDULE',
       'index__Activity_2_W_Beoordelen_fraude-SCHEDULE',
       'index__Activity_3_W_Afhandelen_leads-START',
       'index__Activity_3_W_Beoordelen_fraude-START',
       'index__Activity_3_W_Completeren_aanvraag-SCHEDULE',
       'index__Activity_4_A_PREACCEPTED-COMPLETE',
       'index__Activity_4_W_Afhandelen_leads-COMPLETE',
       ...
       'index__Resource_13_11180.0', 'index__Resource_13_11181.0',
       'index__Resource_13_11189.0', 'index__Resource_13_112.0',
       'index__Resource_13_11200.0', 'index__Resource_13_11201.0',
       'index__Resource_13_11202.0', 'index__Resource_13_11203.0',
       'index__Resource_13_11259.0', 'index__Resource_13_other'],
      dtype='object', length=657)

In [24]:
cats

[113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279

In [25]:
# p2_list

In [26]:
# p2_list.shape